In [56]:
from kafka import KafkaProducer, KafkaConsumer
import requests as rq
from bs4 import BeautifulSoup
import os
import time
os.environ['NEWS_API'] = '7618e996973244b8ab7c6812fa9fb182'

In [9]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key = os.environ.get('NEWS_API'))

In [20]:
result = newsapi.get_top_headlines(q='bitcoin',
                          category='business',
                          language='en',
                          country='us')


In [32]:
response = rq.get(result['articles'][0]['url'])
response

<Response [200]>

In [39]:
from newsapi import NewsApiClient
from kafka import KafkaProducer
import json

# Initialize the News API client
newsapi = NewsApiClient(api_key=os.environ.get('NEWS_API'))
global newsapi
# Kafka Producer Configuration
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

In [45]:
def fetch_news_articles(topic: str, category: str):
    newsapi = NewsApiClient(api_key=os.environ.get('NEWS_API'))
    result = newsapi.get_top_headlines(q=topic,
                                       category=category,
                                       language='en')
    
    articles = result.get('articles', [])
    news_data = []

    for article in articles:
        news_data.append({
            'title': article['title'],
            'summary': article['description'],
            'link': article['url']
        })

    return news_data

In [54]:
def send_to_kafka(news_data, topic):
    for article in news_data:
        producer.send(topic, article)
        producer.flush()

def monitor_news(topic, category):
    seen_articles = set()
    
    while True:
        news_articles = fetch_news_articles(topic, category)
        
        new_articles = [article for article in news_articles if article['link'] not in seen_articles]
        if new_articles:
            send_to_kafka(new_articles, topic)
            for article in new_articles:
                seen_articles.add(article['link'])
        
        time.sleep(5)

In [58]:
if __name__ == "__main__":
    topic = 'AI'
    category = 'technology'
    monitor_news(topic, category)
    print("Monitoring news articles and sending to Kafka.")

KeyboardInterrupt: 

In [69]:
res = rq.get('https://sheharyarakhtar.github.io')
BeautifulSoup(res.content, 'html.parser')

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<!--==================== FAVICON ====================-->
<link href="assets/img/memoji.jpg" rel="icon"/>
<!--==================== UNICONS ====================-->
<link href="https://unicons.iconscout.com/release/v4.0.0/css/line.css" rel="stylesheet"/>
<!--==================== SWIPER CSS ====================-->
<link href="assets/css/swiper-bundle.min.css" rel="stylesheet"/>
<!--==================== CSS ====================-->
<link href="assets/css/styles.css" rel="stylesheet"/>
<!--    =========== Tracker =========      -->
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-2NYWM62ET3"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());
        gtag('config', 'G-2NYWM62ET3');
    </script>
<title>Sheh

In [79]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FlinkKafkaConsumer, FlinkKafkaProducer
from pyflink.datastream.functions import MapFunction
from pyflink.common.serialization import SimpleStringSchema
from pyflink.common.typeinfo import Types


In [86]:
class MyMapFunction(MapFunction):
    def map(self, value):
        # Process the data (e.g., transform or enrich)
        print(value)
        return value.upper()  # Example transformation

os.listdir()

['.DS_Store',
 'Untitled.ipynb',
 'producer.py',
 'consumer.py',
 'flink-sql-connector-kafka_2.11-1.13.0.jar',
 '.ipynb_checkpoints']

In [94]:
env = StreamExecutionEnvironment.get_execution_environment()
env.add_jars('./flink-sql-connector-kafka_2.11-1.13.0.jar')
consumer_props = {
    'bootstrap.servers':'localhost:9092',
    'group.id':'flink-group'
}
# kafka_source = FlinkKafkaConsumer(
#     topics=['news-topic-ai','news-topic-de'],
#     deserialization_schema=SimpleStringSchema(),
#     properties=consumer_props
# )

Py4JJavaError: An error occurred while calling None.java.net.URL.
: java.net.MalformedURLException: no protocol: ./flink-sql-connector-kafka_2.11-1.13.0.jar
	at java.net.URL.<init>(URL.java:610)
	at java.net.URL.<init>(URL.java:507)
	at java.net.URL.<init>(URL.java:456)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.flink.api.python.shaded.py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at org.apache.flink.api.python.shaded.py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at org.apache.flink.api.python.shaded.py4j.Gateway.invoke(Gateway.java:238)
	at org.apache.flink.api.python.shaded.py4j.commands.ConstructorCommand.invokeConstructor(ConstructorC

In [ ]:
StreamExecutionEnvironment.add_jars()